In [ ]:
from helper import load_metadata_text
from agents import run_python_da_agent, run_data_scientist_agent, run_orchestrator_agent, run_summarize_agent, run_all_agents
metadata_text = load_metadata_text()

# Input:
## user question: `prompt`

In [ ]:
prompt = "Analyze the interstate migration pattern of Texas, from 2012 to 2022."
results = run_all_agents(original_prompt=prompt, metadata_text= metadata_text, verbose = True)

In [ ]:
print(results['summary'])

In [ ]:
results['report'].keys()

In [ ]:
results['report']

## Customized Input: Focus

In [ ]:
prompt = '''
Compute the 5-year median net-migration rate, from 2018 to 2022, of Minnesota for all age and agi classes. 
Compute the 5-year median (during the same time) of the net-migration rates for the Midwest Region for the same classes. 
Compare them by using Minnesota rates subtracted by the midwest rates. Use a heatmap to visualize the difference (Both these rates are in percentage).
'''
focus = "Make the report more tech oriented. The main audience are analysts"

results = run_all_agents(original_prompt=prompt , focus= focus, metadata_text= metadata_text, verbose = False)

# Testing Use Only
The following codes are for testing purposes
## Orchestrator agent

In [ ]:
from agents import build_focus

prompt = "Compare the migration activities between Minnesota and Wisconsin, from 2015 to 2022."
focus = "Pay extra attention to the difference between different age and agi classes."

user_prompt = build_focus(original_question=prompt, focus = focus)
plan = run_orchestrator_agent(user_prompt=prompt, metadata_text=metadata_text)


In [ ]:
plan['plan']

In [ ]:
verbose = True

if verbose:
    print(f"Orchestrator plan:\n{plan['plan']}\n")

if plan['requires_clarification']:
    results = {
        "type": "clarification",
        "question": plan['clarification_question']
    }
else:
    shared_env = {}
    shared_meta = {}
    ds_report = {}
    for step in plan['plan']:
        step_id = step['step_id']
        goal = step['goal']
        da_prompt = step['da_prompt']
        ds_prompt = step['ds_prompt']
        depends_on = step['depends_on']

        if verbose:
            print(f"\n---Executing Step {step_id}: {goal} ---\n")

        if da_prompt is not None:
            if verbose:
                print(f"[DA] Running DA step {step_id} with prompt:\n{da_prompt}\n")

            output = run_python_da_agent(
                user_prompt=da_prompt,
                metadata_text="",
                verbose=False,
            )

        if ds_prompt is not None:
            if verbose:
                print(f"[DS] Running DS step {step_id} with prompt:\n{ds_prompt}\n")
                print(f"[DS] Depends on: {depends_on}")

            # Assemble DS environment containing all needed DA results
            ds_env = {}
            ds_meta = {}
            for dep in depends_on:
                try:
                    ds_env[f"df_{dep}"] = shared_env[f"df_{dep}"]
                    ds_meta[f"df_{dep}"] = shared_meta[f"df_{dep}"]
                except: 
                    pass
            output = run_data_scientist_agent(
                user_prompt=ds_prompt,
                env=ds_env,
                env_meta=ds_meta,
                max_steps = 5,
                verbose=False,
            )

            if verbose:
                print(f"[DS] Output of DS step {step_id}:\n{output.get('answer')}\n")

            ds_report[f"ds_step_{step_id}"] = output.get("answer")
        
        try:
            shared_env[f"df_{step_id}"] = output.get("dataframe")
            shared_meta[f"df_{step_id}"] = output.get("metadata")
        except:
            pass


In [ ]:
run_summarize_agent(ds_report=ds_report, user_prompt = user_prompt, metadata_text= metadata_text)

In [ ]:
prompt = "Compute the out migration rate of MN whose gross income is over $100,000 from 2015 to 2022"
answer = run_python_da_agent(prompt, metadata_text, max_steps=5)
if answer["dataframe"] is not None:
    df1 = answer['dataframe']
    df1_meta = answer['metadata']


In [ ]:
prompt = "Compute the out-migration rate of high-income households (gross income over $100,000) in the Midwest region from 2015 to 2022."
answer = run_python_da_agent(prompt, metadata_text, max_steps=5)
if answer["dataframe"] is not None:
    df2 = answer["dataframe"]
    df2_meta = answer["metadata"]

In [ ]:
prompt = "What is the annual national average out-migration rates for high-income households (gross income over $100,000), from 2015 to 2022?"
answer = run_python_da_agent(prompt, metadata_text, max_steps=5)
if answer["dataframe"] is not None:
    df3 = answer["dataframe"]
    df3_meta = answer["metadata"]

In [ ]:
env = {"df1": df1, "df2": df2, "df3": df3}
env_meta = {"df1": df1_meta, "df2": df2_meta, "df3": df3_meta}

In [ ]:
prompt ='''
Analyze the trend of out-migration rate with of higher income households (gross income over $100,000) in Minnesota, 
and compare it with the corresponding rates in Midwest and the national average. The data is provided in `env`. 
Provide insights on how Minnesota's out-migration rate for high-income households differs from regional and national trends between 2015 and 2022. 
'''
answer = run_data_scientist_agent(user_prompt=prompt, env = env, env_meta=env_meta, metadata_text=metadata_text, max_steps=5, verbose=False)
print(answer['answer'])

In [ ]:
prompt = "Compute the average net migration rates of the Midwest region, from 2018 to 2022, for each age calss and agi class combination."
answer = run_python_da_agent(prompt, metadata_text, max_steps=5)
if answer["dataframe"] is not None:
    data_midwest = answer

prompt = "Compute the average net migration rates of Minnesota, from 2018 to 2022, for each age calss and agi class combination."
answer = run_python_da_agent(prompt, metadata_text, max_steps=5)
if answer["dataframe"] is not None:
    data_mn = answer

In [ ]:
env = {"df_midwest": data_midwest["dataframe"], "df_mn": data_mn["dataframe"]}
env_meta = {"df_midwest": data_midwest["metadata"], "df_mn": data_mn["metadata"]}

In [ ]:
prompt ='''
Compare the net migration rates (use percentage as its unit) between Minnesota and the Midwest region from 2018 to 2022, segmented by age class and AGI class.
Use a heatmap to visualize the differences between Minnesota and the Midwest region across these segments.
'''
answer = run_data_scientist_agent(user_prompt=prompt, env = env, env_meta=env_meta, metadata_text=metadata_text, max_steps=5, verbose=False)
print(answer['answer'])